In [1]:
import os
import h5py
import time
import numpy as np
import seqdatasets
from eugene import preprocess as pp
from eugene import dataload as dl
from eugene import train
from eugene import models
import seqdata as sd
import xarray as xr
from eugene import settings

from eugene.models.zoo import DeepSTARR

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Change this to where you would like to save all your results
os.chdir("/cellar/users/aklie/projects/ML4GLand/issues/41")  # TODO: change this to your own directory
cwd = os.getcwd()
cwd

'/cellar/users/aklie/projects/ML4GLand/issues/41'

In [3]:
h5_file = "/cellar/users/aklie/data/ml4gland/pubs/lee22/DeepSTARR_data.h5"
h5 = h5py.File(h5_file, "r")
h5.keys()
X_train = h5["X_train"][:]
X_valid = h5["X_valid"][:]
X_test = h5["X_test"][:]
Y_train = h5["Y_train"][:]
Y_valid = h5["Y_valid"][:]
Y_test = h5["Y_test"][:]
h5.close()

XR_train = xr.DataArray(X_train, dims=("_sequence", "_ohe", "_length"))
XR_valid = xr.DataArray(X_valid, dims=("_sequence", "_ohe", "_length"))
YR_train = xr.DataArray(Y_train, dims=("_sequence", "_target"))
YR_valid = xr.DataArray(Y_valid, dims=("_sequence", "_target"))
sdata_train = xr.Dataset({"ohe_seq": XR_train, "target": YR_train})
sdata_valid = xr.Dataset({"ohe_seq": XR_valid, "target": YR_valid})

In [ ]:
sdata_train = seqdatasets.deAlmeida22("train")
sdata_val = seqdatasets.deAlmeida22("val")
pp.ohe_seqs_sdata(sdata_train)
pp.ohe_seqs_sdata(sdata_val)
sdata_train["target"] = xr.concat([sdata_train[target_var] for target_var in ['Dev_log2_enrichment', 'Hk_log2_enrichment']], dim="_targets").transpose("_sequence", "_targets")
sdata_val["target"] = xr.concat([sdata_val[target_var] for target_var in ['Dev_log2_enrichment', 'Hk_log2_enrichment']], dim="_targets").transpose("_sequence", "_targets")
sdata_train.load(), sdata_val.load()

Dataset deAlmeida22 Sequences_Train.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Train.txt has already been downloaded.
Zarr file found. Opening existing zarr file.
Dataset deAlmeida22 Sequences_Val.fa has already been downloaded.
Dataset deAlmeida22 Sequences_activity_Val.txt has already been downloaded.
Zarr file found. Opening existing zarr file.


In [4]:
arch = DeepSTARR(
    input_len=249, 
    output_dim=2, 
)
model = models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer="adam",
    optimizer_lr=0.002,
    optimizer_kwargs=dict(weight_decay=1e-6)
)
model

SequenceModule(
  (arch): DeepSTARR(
    (conv1d_tower): Conv1DTower(
      (layers): Sequential(
        (0): Conv1d(4, 246, kernel_size=(7,), stride=(1,), padding=same)
        (1): BatchNorm1d(246, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (4): Conv1d(246, 60, kernel_size=(3,), stride=(1,), padding=same)
        (5): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (8): Conv1d(60, 60, kernel_size=(5,), stride=(1,), padding=same)
        (9): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
        (12): Conv1d(60, 120, kernel_size=(3,), stride=(1,), padding=same)
 

In [9]:
train_dl = sd.get_torch_dataloader(
    sdata_train,
    batch_size=128,
    sample_dims="_sequence",
    variables=["ohe_seq", "target"],
    shuffle=True,
    prefetch_factor=None,
    #transforms={"ohe_seq": lambda x: x.swapaxes(1, 2)}
)

val_dl = sd.get_torch_dataloader(
    sdata_valid,
    batch_size=128,
    sample_dims="_sequence",
    variables=["ohe_seq", "target"],
    shuffle=False,
    prefetch_factor=None,
    #transforms={"ohe_seq": lambda x: x.swapaxes(1, 2)}
)

In [10]:
batch = next(iter(train_dl))
batch["ohe_seq"].shape, batch["target"].shape

(torch.Size([128, 4, 249]), torch.Size([128, 2]))

In [11]:
batch = next(iter(val_dl))
batch["ohe_seq"].shape, batch["target"].shape

(torch.Size([128, 4, 249]), torch.Size([128, 2]))

In [12]:
train.fit(
    model,
    train_dataloader=train_dl,
    val_dataloader=val_dl,
    gpus=1,
    epochs=100,
    early_stopping_patience=10,
    log_dir=settings.logging_dir,
    name="DeepSTARR",
    version="Trial9",
)

No seed set


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type      | Params
-------------------------------------------
0 | arch         | DeepSTARR | 7.7 M 
1 | train_metric | R2Score   | 0     
2 | val_metric   | R2Score   | 0     
3 | test_metric  | R2Score   | 0     
-------------------------------------------
7.7 M     Trainable params
0         Non-trainable params
7.7 M     Total params
30.745    Total estimated model params siz

Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# DONE!

---